## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [10]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import matplotlib.pyplot as plt

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


In [11]:
fs = 125
window_length_s = 8
window_shift_s = 2
window_overlap_s = 6
window_length = window_length_s * fs
window_shift = window_shift_s * fs
window_overlap = window_overlap_s *fs
low_BPM = 40
low_Hz = low_BPM/60 
high_BPM = 240
high_Hz = high_BPM/60 

In [12]:
def LoadTroikaReferanceFile(ref_fl):
    """
    Loads and extracts signals from a troika referance data file.

    Args:
        ref_fls: (str) filepath to a troika .mat file.

    Returns:
        numpy array for reference values.
    """
    ref_data = sp.io.loadmat(ref_fl)['BPM0']
    return ref_data

In [13]:
def BandpassFilter(signal, fs=125, low_Hz=0.67, high_Hz=4):
    """
    Bandpass filter the signal between 40 & 240 BPM by default.
    
    Args:
        signal: (array) signal to bandpass.
        fs: (int) sampling frequency of the signal, default is 125.
        lower_BPM: (int) lower frequency to bandpass the filter with in BPM, default is 40.
        higher_BPM: (int) higher frequency to bandpass the filter with in BPM, default is 240.

    Returns:
        Filtered signal in a numpy array.
    """
    b, a = sp.signal.butter(3, (low_Hz, high_Hz), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)


In [14]:
def Featurize(signal, fs, NFFT, noverlap):
    """Calculate spectrum in a given time window width

    Args:
        signal: (np.array) The input signal
        fs: (number) The sampling rate of <signal>
        NFFT: (number) The number used for the FFT block.
        noverlap: (number) The number of sample that overlap with previous block
    Returns:
        spectrums: (2-D np.array) The list of FFT results.
                   Columns are corresnponding to FFT results.
        freqs: (1-D np.array) The frequences calculated
               by given <fs> and <NFFT> values
    """

    n = len(signal)

    # Each block has (NFFT - noverlap) non overlapped samples,
    step = (NFFT - noverlap)
    n_block = (n - NFFT) // step + 1

    specs = []
    for i in range(n_block):
        w = signal[i * step:i*step+NFFT].copy()

        # the `plt.specgram` apply hanning window on each stride windows
        # w *= np.hanning(len(w))

        fft = np.abs(np.fft.rfft(w))
        specs.append(fft)

    specs = np.stack(specs, axis=1)
    freq = np.fft.rfftfreq(NFFT, d=1/fs)
    return specs, freq



In [15]:
def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_data = LoadTroikaReferanceFile(ref_fl)
    # Bandpass all the signals
    ppg = BandpassFilter(ppg)
    accx = BandpassFilter(accx)
    accy = BandpassFilter(accy)
    accz = BandpassFilter(accz)
    
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    

    # Compute pulse rate estimates and estimation confidence.
    errors = []
    confidence = []
    

    # Find dominant frequencies for the accelerometer data
 
    # Define the width of a peak and the number of frequencies to consider per estimate
     # A peak as a unique frequency (epsilon=0) grants the best results
    n_best_freqs = 7  # This parameter is also defined by trial and error

    # Calculate the spectogram of the ppg signal on the previously defined windows
    acc_spec, acc_freq = Featurize(acc, fs, NFFT=window_length, noverlap=window_overlap)
    ppg_spec, ppg_freqs = Featurize(ppg, fs, NFFT=window_length, noverlap=window_overlap)

    # Calculate the heart rate estimates
    hr_estimates = np.array([])
    for i in range(len(ppg_spec[0,])):
        hr_estimates = np.append(hr_estimates, Estimates(ppg_spec, ppg_freqs, hr_estimates, n_best_freqs))
    
    # Try to separate arm motion noise from the actual heart beat frequencies
 
    # Calculate error and confidence
    errors = np.absolute(hr_estimates - ref_data.flatten())  # we calculated our estimates in the same windows as the reference
    confidence = np.array([])
    e=0
    for i, estimate in enumerate(hr_estimates):
        confidence = np.append(confidence, CalcConfidence(estimate, ppg_spec[:, i], ppg_freqs, e))

    return errors, confidence

In [16]:
def Estimates(ppg_spec, ppg_freqs, hr_estimates, n_best_freqs):
    """
    Calculate a heart rate estimate from the spectrum of a fraction of a signal
        while taking into account previous estimates.
    
    Args:
        ppg_spec: (array) spectogram of the ppg signal the estimates are made on.
        ppg_freqs: (array) frequencies used to calculate the spectogram of the ppg signal.
        hr_estimates: (array) array containing the previous heart rate estimates.
        n_best_freqs: (int) number of the best frequencies to consider for the estimate.
            The closest of those frequencies to the previous estimate is chosen.

    Returns:
        An estimate in BPM of the heart rate frequency for the given spectrum.
    """
    
    index = len(hr_estimates)
    if index != 0:
        last_estimate = hr_estimates[-1]/60
        candidates = np.argsort(ppg_spec[:, index], axis=0)[-n_best_freqs:]
        estimate = ppg_freqs[candidates[np.argmin(np.absolute(ppg_freqs[candidates] - last_estimate))]] * 60
    else:
        estimate = ppg_freqs[np.argmax(ppg_spec[:,0], axis=0)] * 60
    return estimate


In [17]:

def CalcConfidence(hr_bpm, fft, freqs, e=0):
    """
    This modified SNR algorithm calculates the confidence we have in one estimated heart rate.

    Args:
        hr_bpm: (int) heart rate estimation in BPM to calculate the confidence of.
        fft: (array) spectogram of the signal the estimate was made on.
        freqs: (array) frequencies used to calculate the spectogram.
        epsilon: (int) corresponds to half of the width of a peak in number of BPMs
            (if epsilon=0, the peak is a unique frequency).

    Returns:
        numpy float for the signal to noise ratio which is our confidence value.
    """
    # BPM to Hz
    
    hr_freq = hr_bpm/60

    # Find the frequencies to compare to the entire spectrum
    hr_borders = (hr_freq-e, hr_freq+e)
    hr_index = np.where((freqs>=hr_borders[0]) & (freqs<=hr_borders[1]))

    # Compute signal power and noise power
    power = np.sum(np.abs(fft[hr_index]))/np.sum(np.abs(fft))

    return power

In [18]:
Evaluate()

13.719820925654721

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 